In [99]:
# criamos funçoes validar 
# modificamos asserts no nw
# verificar lista aminoacidos


def validar_dna(seq:str)->bool:
    assert type(seq)==str,'A sequência não foi fornecida em formato de string'
    lista_nucleotidos = ['A','C','T','G']
    for base in seq.upper():
        if base not in lista_nucleotidos:
            return False
    
    return True

def validar_rna(seq:str)->bool:
    assert type(seq)==str,'A sequência não foi fornecida em formato de string'
    lista_nucleotidos = ['A','C','U','G']
    for base in seq.upper():
        if base not in lista_nucleotidos:
            return False
    
    return True


def validar_prot(seq:str)->bool:
    assert type(seq)==str,'A sequência não foi fornecida em formato de string'
    lista_aminoacidos = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y','_']
    for aa in seq.upper():
        if aa not in lista_aminoacidos:
            return False
    
    return True

def formatar(x):
    if type(x) is int:
        return f"{x:>3d}"
    else:
        return f"{x:>3}"
    
def print_matrix(S1,S2,M):
    
    print(*list(" " + S1), sep="   ")
    for x2, L in zip(S2,M): #x2 caracter que vem da string S2
        LL=[formatar(x) for x in L]
        print(x2,*LL)
    print()

def score_sub_acid_nuc(x1:str,x2:str,match,mismatch)->int:
    if x1==x2:return match
    else: return mismatch

def score_subst_prot(nome_matriz:str, aa1:str, aa2:str) -> int:
    """
    Esta função abre e lê um destes ficheiros de texto: "BLOSSOM62.txt", "BLOSSOM80.txt", "PAM30.txt", "PAM70.txt" 
    retirando a matriz de substituição do ficheiro como uma matriz bidimensional.
    Depois navega na matriz de substituição obtida e retorna o score correspondente ao ????alinhamento???? de dois dos seguintes caracteres: 
    'A','R','N','D','C','Q','E','G','H','I','L','K','M','F','P','S','T','W','Y','V','B','J','Z','X','_',
    sendo que as letras representam os aminoácidos e o caracter '_' representa o codão stop.
    
    Parameters
    ----------
    nome_matriz:str
        Nome da matriz de substituição que se pretende que seja lida e utilizada para fornecer os scores de substituição

    aa1:str
        Aminoácido que vai ser procurado ao longo da primeira coluna da matriz de substituição

    aa2:str
        Aminoácido que vai ser procurado ao longo da primeira linha da matriz de substituição

    Returns
    -------
    score:int
        Score obtido para o alinhamento dos dois aminoácidos na matriz de substituição

    Raises
    ------
    AssertionError
        Quando o nome da matriz fornecido não corresponde a uma das matrizes aceites pela função:
        "BLOSSOM62", "BLOSSOM80", "PAM30", "PAM70"

    AssertionError
        Quando o caracter inserido correspondente ao 'aa1' não é válido como aminoácido ou codão stop

    AssertionError
        Quando o caracter inserido correspondente ao 'aa2' não é válido como aminoácido ou codão stop
    """
    assert nome_matriz.upper() in ["BLOSSOM62", "BLOSSOM80", "PAM30", "PAM70"],"A matriz de substituição escolhida é inválida, apenas aceita as matrizes 'BLOSSOM62', 'BLOSSOM80', 'PAM30', 'PAM70'"
    with open(nome_matriz.upper()+".txt", "r") as file:
        matriz = []
        for line in file:
            line = line.strip()
            linha = []
            valor = ""
            for elem in line:
                if elem == " ":
                    if len(valor) != 0:
                        linha.append(valor)
                        valor = ""
                else:
                    valor += elem
            linha.append(valor)
            matriz.append(linha)
    aa1, aa2 = aa1.upper(), aa2.upper()
    lista_aminoacidos = ['A','R','N','D','C','Q','E','G','H','I','L','K','M','F','P','S','T','W','Y','V','B','J','Z','X','_']
    assert aa1 in lista_aminoacidos,f"O caracter {aa1} não é válido como aminoácido ou codão stop"
    assert aa2 in lista_aminoacidos,f"O caracter {aa2} não é válido como aminoácido ou codão stop"
    coluna, linha = 0, 0
    for i in range(1,len(matriz)):
        if aa1 == matriz[i][0]:
            linha = i
            break
    for pos, elem in enumerate(matriz[0]):
        if elem == aa2:
            coluna = pos
            break
    score = matriz[linha][coluna]
    return int(score)

def reconstrucao(seq1,seq2,trace):
    aligned_seq1 = ''
    aligned_seq2 = ''
    linha=len(seq1)-1
    coluna=len(seq2)-1

    while trace[linha][coluna]!=0:
        if trace[linha][coluna] == '↖':
            aligned_seq1 = seq1[linha] + aligned_seq1
            aligned_seq2 = seq2[coluna] + aligned_seq2
            linha -= 1
            coluna -= 1
        elif trace[linha][coluna] == '↑':
            aligned_seq1 = seq1[linha] + aligned_seq1
            aligned_seq2 = '-' + aligned_seq2
            linha -= 1
        elif trace[linha][coluna] == '←':
            aligned_seq1 = '-' + aligned_seq1
            aligned_seq2 = seq2[coluna] + aligned_seq2
            coluna-= 1
    print(aligned_seq1,aligned_seq2,sep='\n')
    return (aligned_seq1,aligned_seq2)

def needleman_wunsch(seq1:str,seq2:str,seq_tipo:str,gap:int=-4,matriz:str='Blossom62',match:int=2,mismatch:int=-1,matrizes=True ):
    """
    Realiza o alinhamento global de duas sequências recorrendo o algoritmo Needleman-Wunsch.

    Parameters
    ----------
    seq1 : str
      Primeira sequência
    seq2 : str
      Segunda sequência

    seq_tipo: str
        Identifica o tipo das sequências que se pretende alinhar

    gap: int
        Valor a utilizar para penalizar os espaçamentos. Por padrão considera-se o valor de penalização como -4

    matriz: str
        Matriz de substituição de aminoacidos a utilizar para realizar o alinhamento de proteínas

    match: int
        Valor a utilizar para quando as duas bases nucleicas são iguais. Por default é 2. Apenas para o alinhamento de DNA/RNA

    mismatch: int
        Valor a utilizar para quando as duas bases nucleicas são diferentes. Por default é -1. Apenas para o alinhamento de DNA/RNA

    matrizes: bool
        Indica 

    Returns
    ----------
    Tuple
     devolve um tuplo com a matriz de pontuação, a matriz de rastreamento e o alinhamento ótimo
    
    """
    assert seq_tipo.upper() in ['DNA','RNA','PROTEIN'], 'O tipo de sequência fornecido não é valido, apenas pode assumir o valor de DNA, RNA ou protein'
    
    if seq_tipo.upper()=='DNA': 
        assert validar_dna(seq1),'A sequencia 1 não é DNA'
        assert validar_dna(seq2),'A sequencia 2 não é DNA'  

    elif seq_tipo.upper()=='RNA':
        assert validar_rna(seq1),'A sequencia 1 não é RNA'
        assert validar_rna(seq2),'A sequencia 2 não é RNA'

    else:
        assert validar_prot(seq1), 'A sequencia 1 não é uma proteina'
        assert validar_prot(seq2), 'A sequencia 2 não é uma proteina'

    if seq1 == "" or seq2 == "":
        raise TypeError("A sequência fornecida não pode ser uma string vazia")


    seq1='-'+seq1
    seq2='-'+seq2
    nrows = len(seq1)
    ncols = len(seq2)

    score_matrix=[[0]*ncols for linhas in range(nrows)] # iniciar a matriz score apenas com 0
    score_matrix[0]=[c*gap for c in range(ncols)] #prenchimento da primeira linha da matriz score
    for linha in range(1, nrows):score_matrix[linha][0] = linha * gap # preenchimento da primeira coluna da matriz score

    traceback_matrix = [[0] * ncols for linhas in range(nrows)] # iniciar a matriz traceback com 0
    traceback_matrix[0] =[0 if col==0 else '←' for col in range(ncols)] # prenchimento da primeira linha da matriz traceback
    for linha in range(1,nrows): traceback_matrix[linha][0]='↑' # preenchimento da primeira coluna da matriz traceback

    if seq_tipo.upper()=='DNA' or seq_tipo.upper()=='RNA': #alinhamento de acidos nucleicos
      for linha in range(1, nrows):
        for coluna in range(1, ncols):
          diagonal = score_matrix[linha - 1][coluna - 1] + score_sub_acid_nuc(seq1[linha],seq2[coluna],match,mismatch)
          cima = score_matrix[linha - 1][coluna] + gap
          esquerda = score_matrix[linha][coluna - 1] + gap

          max_score = max(diagonal, esquerda, cima)
          score_matrix[linha][coluna] = max_score

          if max_score == diagonal: traceback_matrix[linha][coluna] = '↖'
          elif max_score == cima: traceback_matrix[linha][coluna] = '↑'
          elif max_score == esquerda: traceback_matrix[linha][coluna] = '←'

    else: # alinhamento de proteinas
      for linha in range(1, nrows):
        for coluna in range(1, ncols):
          diagonal = score_matrix[linha - 1][coluna - 1] + score_subst_prot(matriz,seq1[linha],seq2[coluna])
          cima = score_matrix[linha - 1][coluna] + gap
          esquerda = score_matrix[linha][coluna - 1] + gap

          max_score = max(diagonal, esquerda, cima)
          score_matrix[linha][coluna] = max_score

          if max_score == diagonal: traceback_matrix[linha][coluna] = '↖'
          elif max_score == cima: traceback_matrix[linha][coluna] = '↑'
          elif max_score == esquerda: traceback_matrix[linha][coluna] = '←'

    if matrizes:
        print("Matriz de Pontuação:")
        print_matrix(seq2,seq1,score_matrix)
        print("Matriz de Rastreamento:")
        print_matrix(seq2,seq1,traceback_matrix)
        return score_matrix[-1][-1],reconstrucao(seq1,seq2,traceback_matrix)
    
    else:
        return score_matrix[-1][-1],reconstrucao(seq1,seq2,traceback_matrix)

In [100]:
# exemplo de alinhamento de DNA
needleman_wunsch("PHSWG","HGWAG","PROTEIN")

FileNotFoundError: [Errno 2] No such file or directory: 'BLOSSOM62.txt'

In [87]:
from contextlib import AbstractContextManager
from typing import Any
import unittest

class Testneedlemanwunch(unittest.TestCase):
    def test_nw_seq_tipo_caracter_errado_DNA(self):
        with self.assertRaises(AssertionError):
            needleman_wunsch("ACTGG", "AGA","DNA/") 
    
    def test_nw_seq_tipo_caracter_errado_RNA(self):
        with self.assertRaises(AssertionError):
            needleman_wunsch("ACUGG", "AUA","RNA/")
    
    def test_nw_seq_tipo_caracter_errado_PROTEIN(self):
        with self.assertRaises(AssertionError):
            needleman_wunsch("CREATINNA", "BEEFFAA","/PROTEIN") 
    
    def test_nw_caracter_seq1_DNA_errado(self):
        with self.assertRaises(AssertionError):
            needleman_wunsch("ACTGG-", "AGA","DNA")

    def test_nw_caracter_seq2_DNA_errado(self):
        with self.assertRaises(AssertionError):
            needleman_wunsch("ACTGG", "AGA%","DNA")

    def test_nw_caracter_seq1_RNA_errado(self):
        with self.assertRaises(AssertionError):
            needleman_wunsch("ACUGG-", "AUA","RNA")

    def test_nw_caracter_seq2_RNA_errado(self):
        with self.assertRaises(AssertionError):
            needleman_wunsch("ACUGG", "AUA/","RNA")

    def test_nw_caracter_seq1_PROTEINA_errado(self):
        with self.assertRaises(AssertionError):
            needleman_wunsch("CREATINNA%", "BEEFFAA","PROTEIN")

    def test_nw_caracter_seq2_PROTEINA_errado(self):
        with self.assertRaises(AssertionError):
            needleman_wunsch("CREATINNA", "BEEFFAA%/","PROTEIN")
    
    def test_nw_seq1_not_dna(self):
        with self.assertRaises(AssertionError):
            needleman_wunsch("ACUGG", "AGA","DNA")
    
    def test_nw_seq2_not_dna(self):
        with self.assertRaises(AssertionError):
            needleman_wunsch("ACUGG", "AUA","DNA")
    
    def test_nw_seq1_not_rna(self):
        with self.assertRaises(AssertionError):
            needleman_wunsch("ACTGG", "AUA","RNA")
    
    def test_nw_seq2_not_rna(self):
        with self.assertRaises(AssertionError):
            needleman_wunsch("ACUGG", "ATA","RNA")
    
    def test_seq1_vazio(self):
        with self.assertRaises(TypeError):
            needleman_wunsch("", "ATA","DNA")

    def test_seq2_vazio(self):
        with self.assertRaises(TypeError):
            needleman_wunsch("ACTGG","","DNA")

    def test_needleman_wunsch_letras_maiusculas(self):
        score, _ = needleman_wunsch("ACTGG", "AGA", "DNA")
        self.assertEqual(score, -5)

    def test_needleman_wunsch_letras_minusculas(self):
        score, _ = needleman_wunsch("actgg", "aga", "dna")
        self.assertEqual(score, -5)

    def test_alinhamento_DNA_correto(self):
        self.assertEqual(needleman_wunsch("AGATT","ACTA","DNA"),(-2, ('AGATT', 'A-CTA')))

    def test_alinhamento_RNA_correto(self):
        self.assertEqual(needleman_wunsch("UACAA","AGAU","RNA"),(-2, ('UACAA', '-AGAU')))
    
    def test_alinhamento_PROTEIN_correto(self):
        self.assertEqual(needleman_wunsch("AUC","FUADCD","PROTEIN"),(, ('AUC', 'FUADCD')))

suite = unittest.TestLoader().loadTestsFromTestCase(Testneedlemanwunch)
unittest.TextTestRunner(verbosity=3).run(suite)

test_matriz_correta (__main__.Testneedlemanwunch.test_matriz_correta) ... ok
test_matriz_errada (__main__.Testneedlemanwunch.test_matriz_errada) ... ok
test_needleman_wunsch_letras_maiusculas (__main__.Testneedlemanwunch.test_needleman_wunsch_letras_maiusculas) ... ok
test_needleman_wunsch_letras_minusculas (__main__.Testneedlemanwunch.test_needleman_wunsch_letras_minusculas) ... ok
test_nw_caracter_seq1_DNA_errado (__main__.Testneedlemanwunch.test_nw_caracter_seq1_DNA_errado) ... ok
test_nw_caracter_seq1_PROTEINA_errado (__main__.Testneedlemanwunch.test_nw_caracter_seq1_PROTEINA_errado) ... ok
test_nw_caracter_seq1_RNA_errado (__main__.Testneedlemanwunch.test_nw_caracter_seq1_RNA_errado) ... ok
test_nw_caracter_seq2_DNA_errado (__main__.Testneedlemanwunch.test_nw_caracter_seq2_DNA_errado) ... ok
test_nw_caracter_seq2_PROTEINA_errado (__main__.Testneedlemanwunch.test_nw_caracter_seq2_PROTEINA_errado) ... ok
test_nw_caracter_seq2_RNA_errado (__main__.Testneedlemanwunch.test_nw_caracter_

Matriz de Pontuação:
    -   A   G   A
-   0  -4  -8 -12
A  -4   2  -2  -6
C  -8  -2   1  -3
T -12  -6  -3   0

Matriz de Rastreamento:
    -   A   G   A
-   0   ←   ←   ←
A   ↑   ↖   ←   ↖
C   ↑   ↑   ↖   ↖
T   ↑   ↑   ↖   ↖

ACT
AGA
Matriz de Pontuação:
    -   A   G   A
-   0  -4  -8 -12
A  -4   2  -2  -6
C  -8  -2   1  -3
T -12  -6  -3   0

Matriz de Rastreamento:
    -   A   G   A
-   0   ←   ←   ←
A   ↑   ↖   ←   ↖
C   ↑   ↑   ↖   ↖
T   ↑   ↑   ↖   ↖

ACT
AGA
Matriz de Pontuação:
    -   A   G   A
-   0  -4  -8 -12
A  -4   2  -2  -6
C  -8  -2   1  -3
T -12  -6  -3   0
G -16 -10  -4  -4
G -20 -14  -8  -5

Matriz de Rastreamento:
    -   A   G   A
-   0   ←   ←   ←
A   ↑   ↖   ←   ↖
C   ↑   ↑   ↖   ↖
T   ↑   ↑   ↖   ↖
G   ↑   ↑   ↖   ↖
G   ↑   ↑   ↖   ↖

ACTGG
A--GA
Matriz de Pontuação:
    -   a   g   a
-   0  -4  -8 -12
a  -4   2  -2  -6
c  -8  -2   1  -3
t -12  -6  -3   0
g -16 -10  -4  -4
g -20 -14  -8  -5

Matriz de Rastreamento:
    -   a   g   a
-   0   ←   ←   ←
a   ↑   ↖  

<unittest.runner.TextTestResult run=19 errors=0 failures=0>

# TESTE MOTIFS

In [101]:
def alfabeto(tipo_seq:str)->str:
    '''
    Esta função recebe o tipo de uma sequência e devolve o seu respetivo alfabeto

    Parameters
    ---------
    tipo_seq:str
        Tipo válido de uma sequência biológica. Pode assumir os valores de DNA, RNA, protein,
        proteína ou prot

    Returns
    -------
    list(str)
        Composto pelo tipo de sequência e o seu respetivo alfabeto
    
    Raises
    ------
    ValueError
        Quando o tipo de sequência fornecido não é um tipo de sequência biológica válida
    '''
    from unidecode import unidecode
    if tipo_seq.upper()=="DNA":
        DNA="ACGT"
        return [tipo_seq.upper(),DNA]
    elif tipo_seq.upper()=="RNA":
        RNA="ACGU"
        return [tipo_seq.upper(),RNA]
    elif unidecode(tipo_seq.upper()) in ["PROTEIN","PROTEINA","PROT"]:
        protein="ABCDEFGHIKLMNPQRSTVWYZ"
        return ["Proteína", protein]
    else:
        raise ValueError(f"O tipo {tipo_seq} não é válido. Os tipos de cadeia válida são: DNA, RNA ou proteina") 

def pwm (seqs: list, tipo_seq: str,pseudo=0)->list(dict[str,float]):
    '''
    Esta função recebe uma lista de sequências alinhadas e devolve uma matriz desingada de position weigth matrix (PWM), onde se representa
    a probabilidade de cada letra de um dado alfabeto biológico estar presente numa determinada posição.

    Parameters
    ----------
    seqs:list(str)
        Contêm um conjunto de sequências alinhadas em formato de string

    tipo_seq:str
        Tipo da sequencia que se pretende realiazar a PWM. Pode assumir o valor de DNA, RNA, protein, proteína ou prot

    pseudo: int ou float
        Valor a utilizar para realizar a pseudocontagem. Por padrão o valor é 0

    Returns
    -------
    PWM:list(dict)
        Valor da probabilidade de cada letra do alfabeto estar presente em cada posição das sequências
    
    Raises
    ------
    AssertionError
        Quando um elemento de uma das sequências fornecidas não está presente no alfabeto utilizado

        Quando o tamanho das sequências fornecidas é diferente
    '''
    alfa=alfabeto(tipo_seq)
    
    tamanho=[]
    
    for I,seq in enumerate(seqs):
        tamanho.append(len(seq))
        for pos,elem in enumerate(seq):
            assert elem in alfa[1],f"Na sequencia {seq}, o elemento {elem} na posição {pos+1} não se encontra no alfabeto do {alfa[0]}"
        assert tamanho[I]==tamanho[I-1],f"A sequência {seq} não possui o mesmo tamanho que as restantes"

    elementos_coluna=list(zip(*seqs))

    PWM=[]

    for coluna in elementos_coluna:
        contagem={}
        for elem in alfa[1]:
            contagem[elem]=(coluna.count(elem) + pseudo)/(len(seqs)+len(alfa[1])*pseudo)
        
        PWM.append(contagem)  
    
    return PWM

def prob_seq (seq: str, pwm: list(dict[str,float])):
    '''
    Esta função calcula a probabilidade de uma determinada sequencia ter sido gerada pela PWM

    Parameters
    ---------
    seq:str
        Sequência que se pretende calcular a probabilidade de ter sido gerada pela PWM
    
    pwm:list(dict[str,floar])
        Corresponde às probabilidades de cada elemento aparecer nas várias posições da sequência

    Returns
    -------
    prod:float
        Probabilidade de a sequencia fornecida ter sido gerada pela PWM

    Raises
    ------
    AssertionError
        Quando o tamanho da sequência não é igual ao tamanho da PWM

    '''
    assert len(seq)<=len(pwm),f"A sequência {seq} não é do mesmo tamanho que a PWM"

    prod=1
    for I,elem in enumerate(seq):
        prod=prod*pwm[I][elem]
    return prod

def seq_mais_provavel(seq,pwm,resultado="Max"):
 '''
  Esta função recebe uma sequência e utiliza a PWM para determinar qual a subsequência mais provável de ter sido gerada pela PWM

  Parameters
  --------
  seq: str
    sequência na qual se pretende analisar a probablidade de cada subsequência ter sido gerada pela PWM

  pwm: list(dict[str,float])
    PWM de um determinado alinhamento

  Resultado: str
    Corresponde ao valor que se pretende obter, por exemplo, quando assume o valor "Máximo" devolve a(s) subsequência(s) 
    com maior probabilidade de ter sido gerada pela PWM, quando assume o valor mínimo devolve a(s) sequência(s) com menor probabildade
    de serem formadas pela PWM

  Returns
  -------
  str
    Corresponde à sequência de maior/menor probabilidade
 '''

 assert len(seq)>=len(pwm),f'''
 A sequência {seq} não pode ser utilizada para determinar a sequência mais próvavel. A sequência deve ter, no mínimo, o mesmo tamnho da PWM
 '''
 w=len(pwm)
 snips=[seq[P:P+w] for P in range(len(seq)-w+1)]
 res={}
 for snip in snips:
    prob=prob_seq(snip,pwm)
    res[snip]=prob
  
 if resultado.upper() in ['MAX','MAXIMO']:
    return max(res, key=res.get)
 
 if resultado.upper() in ['MIN','MINIMO']:
     return min(res, key=res.get)
 
def pssm(seqs,tipo_seq,pseudo=1):
    import math
    alfa=alfabeto(tipo_seq)
    tamanho=[]
    for I,seq in enumerate(seqs):
        tamanho.append(len(seq))
        for pos,elem in enumerate(seq):
            assert elem in alfa[1],f"Na sequencia {seq}, o elemento {elem} na posição {pos+1} não se encontra no alfabeto do {alfa[0]}"
        assert tamanho[I]==tamanho[I-1],f"A sequência {seq} não possui o mesmo tamanho que as restantes"

    elementos=list(zip(*seqs))

    PSSM=[{b:math.log2(((elemento.count(b)+pseudo)/(len(seqs)+len(alfa[1])*pseudo))/(1/len(alfa[1]))) for b in alfa[1]}  for elemento in elementos]

    return PSSM


In [119]:
pssm(("AGC","AGA","AGT"),"DNA")

[{'A': 1.1926450779423958,
  'C': -0.8073549220576043,
  'G': -0.8073549220576043,
  'T': -0.8073549220576043},
 {'A': -0.8073549220576043,
  'C': -0.8073549220576043,
  'G': 1.1926450779423958,
  'T': -0.8073549220576043},
 {'A': 0.19264507794239583,
  'C': 0.19264507794239583,
  'G': -0.8073549220576043,
  'T': 0.19264507794239583}]

In [115]:
"""seq_mais_provavel("AGC", [
    {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0},
    {'A': 0.0, 'C': 0.0, 'G': 1.0, 'T': 0.0},
    {'A': 0.3333333333333333, 'C': 0.3333333333333333, 'G': 0.0, 'T': 0.3333333333333333}
], resultado="MAX")"""

seq_mais_provavel("AGC", [
    {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0},
    {'A': 0.0, 'C': 0.0, 'G': 1.0, 'T': 0.0},
    {'A': 0.3333333333333333, 'C': 0.3333333333333333, 'G': 0.0, 'T': 0.3333333333333333}
], resultado="MIN")
"""prob_seq("AGC",[{'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0},
 {'A': 0.0, 'C': 0.0, 'G': 1.0, 'T': 0.0},
 {'A': 0.5, 'C': 0.25, 'G': 0.0, 'T': 0.25}])"""
#pwm (("AGC","AGA","AGT","AGA"),"DNA")
#prob_seq("AGC",pwm(("AGC","AGA","AGT","AGA"),"DNA"))

'prob_seq("AGC",[{\'A\': 1.0, \'C\': 0.0, \'G\': 0.0, \'T\': 0.0},\n {\'A\': 0.0, \'C\': 0.0, \'G\': 1.0, \'T\': 0.0},\n {\'A\': 0.5, \'C\': 0.25, \'G\': 0.0, \'T\': 0.25}])'

In [123]:
import unittest

class TestMOTIFS(unittest.TestCase):
    
    def test_alfabeto_caracter_errado_dna(self):
        with self.assertRaises(ValueError):
            alfabeto("DNA%")

    def test_alfabeto_caracter_errado_rna(self):
        with self.assertRaises(ValueError):
            alfabeto("RNA/")

    def test_alfabeto_caracter_errado_protein(self):
        with self.assertRaises(ValueError):
            alfabeto("protein@")

    def test_alfabeto_dna_valido(self):
        self.assertEqual(alfabeto("DNA"), ["DNA", "ACGT"])

    def test_alfabeto_rna_valido(self):
        self.assertEqual(alfabeto("RNA"), ["RNA", "ACGU"])

    def test_alfabeto_protein_valido(self):
        self.assertEqual(alfabeto("protein"), ["Proteína", "ABCDEFGHIKLMNPQRSTVWYZ"])

    def test_pwm_caracter_invalido_DNA(self):
        with self.assertRaises(AssertionError):
            pwm(("ACAT", "AGCP", "TGGT"), "DNA")
    
    def test_pwm_caracter_invalido_RNA(self):
        with self.assertRaises(AssertionError):
            pwm(("A&AU", "AGCU", "AUAU"), "RNA")

    def test_pwm_caracter_invalido_protein(self):
        with self.assertRaises(AssertionError):
            pwm(("ACMN", "TGCC", "WYZ/"), "protein")
    
    def test_pwm_tamanhos_diferentes(self):
        with self.assertRaises(AssertionError):
            pwm(("ACGT", "TGCA", "ACG"), "DNA")

    def test_pwm_valid(self):
        self.assertEqual(pwm(("AGC", "AGA", "AGT"), "DNA"), [
        {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0},
        {'A': 0.0, 'C': 0.0, 'G': 1.0, 'T': 0.0},
        {'A': 0.3333333333333333, 'C': 0.3333333333333333, 'G': 0.0, 'T': 0.3333333333333333}])

    def test_prob_seq_caracter_errado(self):
        with self.assertRaises(AssertionError):
            prob_seq("AGC/",[{'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0},
            {'A': 0.0, 'C': 0.0, 'G': 1.0, 'T': 0.0},
            {'A': 0.3333333333333333,
            'C': 0.3333333333333333,
            'G': 0.0,
            'T': 0.3333333333333333}] )

    def test_prob_seq_valido(self):
        self.assertEqual(prob_seq("AGC", [{'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0},
            {'A': 0.0, 'C': 0.0, 'G': 1.0, 'T': 0.0},
            {'A': 0.3333333333333333,
            'C': 0.3333333333333333,
            'G': 0.0,
            'T': 0.3333333333333333}]),0.3333333333333333 ) 

    def test_seq_mais_provavel_max_prob(self):  
        self.assertEqual(seq_mais_provavel("AGC", [
        {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0},
        {'A': 0.0, 'C': 0.0, 'G': 1.0, 'T': 0.0},
        {'A': 0.3333333333333333, 'C': 0.3333333333333333, 'G': 0.0, 'T': 0.3333333333333333}
        ], resultado="MAX"),"AGC")
     

    def test_seq_mais_provavel_min_prob(self):
        self.assertEqual(seq_mais_provavel("AGC", [
        {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0},
        {'A': 0.0, 'C': 0.0, 'G': 1.0, 'T': 0.0},
        {'A': 0.3333333333333333, 'C': 0.3333333333333333, 'G': 0.0, 'T': 0.3333333333333333}
        ], resultado="MIN"),"AGC")

    def test_pssm_tamanhos_de_sequencia_diferentes(self):
        with self.assertRaises(AssertionError):
            pssm(("ACT", "TGCA", "TCG"), "DNA")

    def test_pssm_caracter_invalido(self):
        with self.assertRaises(AssertionError):
            pssm(("AC/", "UGC", "UCG"), "RNA")
   
    def test_pssm_valido(self):
        self.assertEqual(pssm(["AGC", "AGA", "AGT"], "DNA"), [
        {'A': 1.1926450779423958, 'C': -0.8073549220576043, 'G': -0.8073549220576043, 'T': -0.8073549220576043},
        {'A': -0.8073549220576043, 'C': -0.8073549220576043, 'G': 1.1926450779423958, 'T': -0.8073549220576043},
        {'A': 0.19264507794239583, 'C': 0.19264507794239583, 'G': -0.8073549220576043, 'T': 0.19264507794239583}
        ])

suite = unittest.TestLoader().loadTestsFromTestCase(TestMOTIFS)
unittest.TextTestRunner(verbosity=3).run(suite)

test_alfabeto_caracter_errado_dna (__main__.TestMOTIFS.test_alfabeto_caracter_errado_dna) ... ok
test_alfabeto_caracter_errado_protein (__main__.TestMOTIFS.test_alfabeto_caracter_errado_protein) ... ok
test_alfabeto_caracter_errado_rna (__main__.TestMOTIFS.test_alfabeto_caracter_errado_rna) ... ok
test_alfabeto_dna_valido (__main__.TestMOTIFS.test_alfabeto_dna_valido) ... ok
test_alfabeto_protein_valido (__main__.TestMOTIFS.test_alfabeto_protein_valido) ... ok
test_alfabeto_rna_valido (__main__.TestMOTIFS.test_alfabeto_rna_valido) ... ok
test_prob_seq_caracter_errado (__main__.TestMOTIFS.test_prob_seq_caracter_errado) ... ok
test_prob_seq_valido (__main__.TestMOTIFS.test_prob_seq_valido) ... ok
test_pssm_caracter_invalido (__main__.TestMOTIFS.test_pssm_caracter_invalido) ... ok
test_pssm_tamanhos_de_sequencia_diferentes (__main__.TestMOTIFS.test_pssm_tamanhos_de_sequencia_diferentes) ... ok
test_pssm_valido (__main__.TestMOTIFS.test_pssm_valido) ... ok
test_pwm_caracter_invalido_DNA (_

test_pwm_tamanhos_diferentes (__main__.TestMOTIFS.test_pwm_tamanhos_diferentes) ... ok
test_pwm_valid (__main__.TestMOTIFS.test_pwm_valid) ... ok
test_seq_mais_provavel_max_prob (__main__.TestMOTIFS.test_seq_mais_provavel_max_prob) ... ok
test_seq_mais_provavel_min_prob (__main__.TestMOTIFS.test_seq_mais_provavel_min_prob) ... ok

----------------------------------------------------------------------
Ran 18 tests in 0.028s

OK


<unittest.runner.TextTestResult run=18 errors=0 failures=0>